In [35]:
import pandas as pd
import bs4
import json
import re
import tqdm

import scipy.stats as st
import numpy as np

import bokeh
from bokeh.plotting import show as show_interactive
from bokeh.plotting import output_file, output_notebook
from bokeh.layouts import column, row
from bokeh.models import CustomJS, TextInput, LassoSelectTool, Select, MultiSelect, ColorBar, Legend, LegendItem
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Button, HTMLTemplateFormatter
from bokeh.events import SelectionGeometry
from bokeh.transform import linear_cmap, jitter
from bokeh.models import FactorRange


In [36]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')
go_df = pd.read_csv('./go_annotations.csv')
kegg_df = pd.read_csv('./kegg_annotations.csv')
ec_df = pd.read_csv('./ec_annotations.csv')

In [37]:
go_df.head()

,GOs,GOs_description,GOs_definition,GOs_obsolete
0,-,-,-,-
1,GO:0000001,mitochondrion inheritance,"The distribution of mitochondria, including th...",False
2,GO:0000002,mitochondrial genome maintenance,The maintenance of the structure and integrity...,False
3,GO:0000003,reproduction,The production of new individuals that contain...,False
4,GO:0000011,vacuole inheritance,The distribution of vacuoles into daughter cel...,False


In [38]:
go_df['GOs_description'].loc[go_df['GOs'] == 'GO:0000001'].values[0]

'mitochondrion inheritance'

In [39]:
kegg_df.head()

,KEGG_ko,KEGG_ko_description
0,-,-
1,ko:K00002,alcohol dehydrogenase (NADP+)
2,ko:K00006,glycerol-3-phosphate dehydrogenase (NAD+)
3,ko:K00011,aldehyde reductase
4,ko:K00012,UDPglucose 6-dehydrogenase


In [40]:
ec_df.head()

,EC,EC_description
0,-,-
1,1.1.1.1,alcohol dehydrogenase
2,1.1.1.105,all-trans-retinol dehydrogenase (NAD(+))
3,1.1.1.141,15-hydroxyprostaglandin dehydrogenase (NAD(+))
4,1.1.1.178,3-hydroxy-2-methylbutyryl-CoA dehydrogenase


In [41]:
# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}

In [42]:
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict

In [43]:
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            

In [44]:
def get_GO_info(go_term):
    
    name = go_df['GOs_description'].loc[go_df['GOs'] == go_term].values[0]
    
    definition = go_df['GOs_definition'].loc[go_df['GOs'] == go_term].values[0]
    
    obsolete = go_df['GOs_obsolete'].loc[go_df['GOs'] == go_term].values[0]
    
    return name, definition, obsolete

In [45]:
get_GO_info('GO:0000001')

('mitochondrion inheritance',
 'The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton.',
 'False')

In [46]:
def get_KEGG_info(term):
    return kegg_df['KEGG_ko_description'].loc[kegg_df['KEGG_ko'] == term].values[0]

In [47]:
get_KEGG_info('ko:K00002')

'alcohol dehydrogenase (NADP+)'

In [48]:
def get_EC_info(term):
    return ec_df['EC_description'].loc[ec_df['EC'] == term].values[0]

In [49]:
get_EC_info('1.1.1.141')

'15-hydroxyprostaglandin dehydrogenase (NAD(+))'

In [50]:
def write_enrichment(lldf, phases, out_prefix, term_column):
    
    with open(f'../enrichment/{out_prefix}_enrichment.txt', 'w') as fl:
    
        for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):

            # print(f'Module {m}')
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, complete_annot, term_column)

            fl.write(f'Module {m}\n')
            for b, fold, t in zip(bs, folds, terms):
                
                if term_column == 'COG_category':
                    
                    definition = COG_dict[t]
                    t = f'{t}: {definition}'
                
                elif term_column == 'GOs':
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
                    else:
                        t = f'{name.capitalize()} ({t}): {definition}'
                        
                elif term_column.split('_')[0] == 'KEGG':
                    name = get_KEGG_info(t)
                    
                    t = f'{t}: {name}'
                    
                elif term_column == 'EC':
                    
                    name = get_EC_info(t)
                    t = f'{t}: {name}'
                    
                        
                # print(f"{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n")

                fl.write(f'{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n')

In [51]:
def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko', 'EC'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            elif tc == 'EC':
                for t in terms:
                    info.append(get_EC_info(t))
                    
            term_df = pd.DataFrame({'module': [m]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold_change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df
        
        
                        
            
                    
            
        
#         for b, fold, t in zip(bs, folds, terms):

#             if term_column == 'GOs':
#                 name, definition, obsolete = get_GO_info(t)
#                 if obsolete:
#                     t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
#                 else:
#                     t = f'{name.capitalize()} ({t}): {definition}'

#             if term_column.split('_')[0] == 'KEGG':
#                 dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')

#                 data = dfs[4]
#                 entry = data.loc[data[0] == 'Entry'][1].values[0]
#                 name = data.loc[data[0] == 'Name'][1].values[0]

#                 t = f'{t}: {name}'

In [52]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')

In [53]:
lldf_nn3 = pd.read_csv('../embedding/test_nn3_leiden_label_df_round_1.csv')

In [54]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_full'].values[0])

26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26


In [55]:
lldf_nn3['leiden_label_full'].unique()

array([31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
       14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [56]:
# %pdb

In [57]:
enrich_full_nn3 = get_enrichment_df(lldf_nn3, 'full', complete_annotation, outfile='./test_nn3_full_enrichment.csv')

100%|██████████| 32/32 [00:18<00:00,  1.72it/s]


In [58]:
enrich_full_nn3 = pd.read_csv('./test_nn3_full_enrichment.csv')

In [59]:
enrich_full_nn3.head(40)

,module,term,info,fold_change,bonferroni
0,0.0,A,RNA processing and modification,2.049872,2.055415e-07
1,0.0,J,"Translation, ribosomal structure and biogenesis",3.709912,2.201301e-38
2,0.0,Q,"Secondary metabolites biosynthesis, transport ...",1.736334,7.081510e-03
3,0.0,C,Energy production and conversion,4.060262,2.463457e-33
4,0.0,E,Amino acid transport and metabolism,2.719741,7.652364e-14
5,0.0,I,Lipid transport and metabolism,2.803674,1.372937e-15
6,0.0,H,Coenzyme transport and metabolism,2.648009,2.239521e-05
7,0.0,F,Nucleotide transport and metabolism,2.901190,3.209168e-05
8,0.0,K,Transcription,1.584613,2.998809e-02
9,0.0,GO:0005739,"Mitochondrion: A semiautonomous, self replicat...",2.625261,1.407122e-04


In [60]:
for term in enrich_full_nn3['term'].unique():
    if 'ko:' not in term and 'GO:' not in term and '.' not in term:
        print(term)

A
J
Q
C
E
I
H
F
K
S
U
Z
T
L
P
O
D
B
Y
M
G


In [61]:
print(max(enrich_full_nn3['module'].values))
print(min(enrich_full_nn3['module'].values))

not_enriched_mod_list = []
enriched_mod_list = []

for num in range(int(max(enrich_full_nn3['module'].values)) + 1):
    if num not in enrich_full_nn3['module'].values:
        not_enriched_mod_list.append(num)
    if num in enrich_full_nn3['module'].values:
        enriched_mod_list.append(num)

print(len(not_enriched_mod_list) + len(enriched_mod_list))
print(max(not_enriched_mod_list + enriched_mod_list))
print()

print(set(range(int(max(enrich_full_nn3['module'].values)) + 1)) - set(enrich_full_nn3['module'].values))
print(len(set(enrich_full_nn3['module'].values)))
print(len(enriched_mod_list))
print(len(not_enriched_mod_list))

31.0
0.0
32
31

{28, 30}
30
30
2


In [62]:
print(not_enriched_mod_list)

[28, 30]


In [63]:
enrich_full_nn3.loc[enrich_full_nn3['module']==72]

,module,term,info,fold_change,bonferroni


In [64]:
enrich_full_nn3['module'].unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 29., 31.])

In [65]:
lldf_nn3['leiden_label_full'].unique()

array([31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
       14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [66]:
lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == 'TTHERM_01386050']

,TTHERM_ID,leiden_label_full
23,TTHERM_01386050,27


In [67]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_full'].values[0])

26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26


In [68]:
len(set(enrich_full_nn3['module'].values))

30

end here. Return to embedding notebook!